In [75]:
from square.client import Client
import os
import pandas as pd

from google_sheets import *

# import pandas as pd
from boto import get_square_secret
import uuid

# import os
import datetime as dt
import pickle

In [76]:
df=pd.read_csv('match_invoice.csv')
df['email']=df['email'].str.lower()
df

,name,match debt,email
0,Aidan Byrne,500,byrneyboii2@gmail.com
1,Alex Dockerill,500,dockerillalex@gmail.com
2,Alex Mansfield,600,aemansfield8@gmail.com
3,Anthony Ball,500,anthonyball89@outlook.com
4,Babajide Onibada,1500,jideonibada@hotmail.com
...,...,...,...
64,Stu Jordan,1000,stuart.jordan86@hotmail.co.uk
65,Theo Horton,1000,theo.horton23@gmail.com
66,Tom Davidson,500,tomdavidson92@gmail.com
67,Will French,1000,william_french@hotmail.com


In [77]:
get_square_secret()

'EAAAFO_dqOOnvRZCLDvOtKfBGznlxPy5zb4anviDPIJBjXjXGkmo5d6w1jxoxtCn'

In [78]:
client = Client(
    access_token=get_square_secret(),
    environment='production')

In [79]:
customers_api = client.customers

In [53]:

result = customers_api.list_customers(
    
)
customers = {str(cust.get('email_address')):cust.get('id') for cust in result.body['customers']}


i=1
while result.cursor:
    print(i)
    i+=1
    result = customers_api.list_customers(
        cursor = result.cursor
    )
    customers2 = {str(cust.get('email_address')):cust.get('id') for cust in result.body['customers']}

    customers.update(customers2)




1
2
3
4
5
6
7
8
9
10


In [68]:
customer_df=pd.DataFrame(data=[{'email':cust, 'id':customers[cust]} for cust in customers])
customer_df['email']=customer_df['email'].str.lower()
customer_df

,email,id
0,none,7QMAV9NTQRNDMVW6V81PKKTKZ4
1,aaronrose1981@gmail.com,TJ6J476K0X5HFCDCXEA8QEN84M
2,aaron.s.c.s@hotmail.co.uk,0VBREWQGYS5G3D5764XW4TZH98
3,83251863.mail@waitrose.co.uk,MPWDP86M290BZ5VWVGK51JMJD8
4,absstacks98@gmail.com,CDWK490YN7HBA3Z3CRRTS843EW
...,...,...
664,oaoviosu@gmail.com,KRVP85KPCH7Y77HPR47JN3WFPC
665,zac.rabanalhunt@gmail.com,224DCGABNQZBAP69A4JM3VBSQ8
666,zachary_mark_williams@yahoo.co.uk,TF1ZAX80D09ST3J7PK4PGY0RN8
667,zakriyagul23@gmail.com,V295YNGJSD4235PXWGMEVKK088


In [69]:
mg = pd.merge(left =df, right = customer_df, how='left', on=['email'])

In [70]:
not_created = mg[mg['id'].isnull()].copy()
not_created

,name,match debt,email,id


In [38]:
for i, row in not_created.iterrows():
    print(row)

    first_name=row['name'].split(' ')[0]
    surname=row['name'].split(' ')[0]

    email=row['email']

    body = {
    'given_name': first_name,
    'family_name': surname,
    'email_address':email,
    
    }

    result = customers_api.create_customer(body)
    print(result)

name                   Zac Rabanal Hunt
match debt                        500.0
email         zac.rabanalhunt@gmail.com
id                                  NaN
Name: 68, dtype: object
<ApiResponse {"customer": {"id": "224DCGABNQZBAP69A4JM3VBSQ8","created_at": "2024-09-24T09:27:41.191Z","updated_at": "2024-09-24T09:27:41Z","given_name": "Zac","family_name": "Zac","email_address": "zac.rabanalhunt@gmail.com","preferences": {"email_unsubscribed": false},"creation_source": "THIRD_PARTY","version": 0}}
>


In [60]:

result = customers_api.list_customers(
    
)
customers = {str(cust.get('email_address')):cust.get('id') for cust in result.body['customers']}


i=1
while result.cursor:
    print(i)
    i+=1
    result = customers_api.list_customers(
        cursor = result.cursor
    )
    customers2 = {str(cust.get('email_address')):cust.get('id') for cust in result.body['customers']}

    customers.update(customers2)




1
2
3
4
5
6
7
8
9
10


In [80]:
customer_df=pd.DataFrame(data=[{'email':cust, 'id':customers[cust]} for cust in customers])
customer_df['email']=customer_df['email'].str.lower()
customer_df

,email,id
0,none,7QMAV9NTQRNDMVW6V81PKKTKZ4
1,aaronrose1981@gmail.com,TJ6J476K0X5HFCDCXEA8QEN84M
2,aaron.s.c.s@hotmail.co.uk,0VBREWQGYS5G3D5764XW4TZH98
3,83251863.mail@waitrose.co.uk,MPWDP86M290BZ5VWVGK51JMJD8
4,absstacks98@gmail.com,CDWK490YN7HBA3Z3CRRTS843EW
...,...,...
664,oaoviosu@gmail.com,KRVP85KPCH7Y77HPR47JN3WFPC
665,zac.rabanalhunt@gmail.com,224DCGABNQZBAP69A4JM3VBSQ8
666,zachary_mark_williams@yahoo.co.uk,TF1ZAX80D09ST3J7PK4PGY0RN8
667,zakriyagul23@gmail.com,V295YNGJSD4235PXWGMEVKK088


In [81]:
mg = pd.merge(left =df, right = customer_df, how='left', on=['email'])
mg=mg.groupby(['name', 'match debt', 'email'])['id'].first().reset_index()

mg

,name,match debt,email,id
0,Aidan Byrne,500,byrneyboii2@gmail.com,ZSBNEMP88D61Z4V4FEM51RHD0W
1,Alex Dockerill,500,dockerillalex@gmail.com,D94PE6KJF55XQAC3KMJ0GMBGNR
2,Alex Mansfield,600,aemansfield8@gmail.com,S3A1PTQ1T90RX9YAHYH2EDY97W
3,Anthony Ball,500,anthonyball89@outlook.com,R6FP8VWM0X5G77GAZP20FKWXK4
4,Babajide Onibada,1500,jideonibada@hotmail.com,CZ3CB8ATNH58BAN5HKEG3W0ESW
...,...,...,...,...
64,Stu Jordan,1000,stuart.jordan86@hotmail.co.uk,WBJGQ6GAE9633AYQQAVE7CM3J8
65,Theo Horton,1000,theo.horton23@gmail.com,5A6A4T4WCN6BKCQD7FCF093N94
66,Tom Davidson,500,tomdavidson92@gmail.com,X3WGSQHHC97PK2KJZ91EBA9HNG
67,Will French,1000,william_french@hotmail.com,BY3NQW9DRX6EF10SFJDHNVEF38


In [82]:
amount_update_factor = 1
catalog_object_id = 'OKAX666RRB7XKNM6BJTRM6IM'

In [83]:
orders_api = client.orders

new_orders = []

for i, row in mg.iterrows():
    print(i)
    amount = row['match debt'] 
    print(amount)
    body = {
        'order': {
            'location_id': 'SBBJSGR24YHYB',
            'reference_id': f'{uuid.uuid1()}',
            'line_items': [
                
                {
                    'quantity': '1',
                    'catalog_object_id': catalog_object_id,
                    'base_price_money': {
                        'amount': amount*amount_update_factor,
                        'currency': 'GBP'
                    }
                }
            ]
        },
        'idempotency_key': f'{uuid.uuid1()}'
    }
    # print(body)
    
    result = orders_api.create_order(body)
    print(result)
    obj_id=result.body['order']['id']

    new_orders.append({'email':row['email'], 'obj_id':obj_id})
    # # print(result)

0
500
<ApiResponse {"order": {"id": "NJHJjp0W7BS60TwRa1f2QPdCOQbZY","location_id": "SBBJSGR24YHYB","line_items": [{"uid": "y6FcCDISkvezg61DI7BN","catalog_object_id": "OKAX666RRB7XKNM6BJTRM6IM","catalog_version": 1727115584321,"quantity": "1","name": "Match Sub","variation_name": "Full Game","base_price_money": {"amount": 500,"currency": "GBP"},"gross_sales_money": {"amount": 500,"currency": "GBP"},"total_tax_money": {"amount": 0,"currency": "GBP"},"total_discount_money": {"amount": 0,"currency": "GBP"},"total_money": {"amount": 500,"currency": "GBP"},"variation_total_price_money": {"amount": 500,"currency": "GBP"},"item_type": "ITEM","total_service_charge_money": {"amount": 0,"currency": "GBP"}}],"created_at": "2024-09-24T09:32:58.720Z","updated_at": "2024-09-24T09:32:58.720Z","state": "OPEN","version": 1,"reference_id": "fc3e47b0-7a57-11ef-9a05-0a06df24a99e","total_tax_money": {"amount": 0,"currency": "GBP"},"total_discount_money": {"amount": 0,"currency": "GBP"},"total_tip_money": {"

In [84]:
order_mg= pd.merge(left = mg, right=pd.DataFrame(new_orders), how= 'left', on = 'email')

order_mg

,name,match debt,email,id,obj_id
0,Aidan Byrne,500,byrneyboii2@gmail.com,ZSBNEMP88D61Z4V4FEM51RHD0W,NJHJjp0W7BS60TwRa1f2QPdCOQbZY
1,Alex Dockerill,500,dockerillalex@gmail.com,D94PE6KJF55XQAC3KMJ0GMBGNR,l7yvqkjPYtv2qd6Gwl2Qfs5NCMWZY
2,Alex Mansfield,600,aemansfield8@gmail.com,S3A1PTQ1T90RX9YAHYH2EDY97W,ZzLOpr7Gy0MOs10DUi2srkvpPnHZY
3,Anthony Ball,500,anthonyball89@outlook.com,R6FP8VWM0X5G77GAZP20FKWXK4,pN0EFuSgruOycd7CnkcVm9WCpGEZY
4,Babajide Onibada,1500,jideonibada@hotmail.com,CZ3CB8ATNH58BAN5HKEG3W0ESW,Rx8uPwF5zHJ7YcpWDQNtIlgWsoIZY
...,...,...,...,...,...
64,Stu Jordan,1000,stuart.jordan86@hotmail.co.uk,WBJGQ6GAE9633AYQQAVE7CM3J8,tBdcEbNOOmXLcYR31iimNZw8tP9YY
65,Theo Horton,1000,theo.horton23@gmail.com,5A6A4T4WCN6BKCQD7FCF093N94,DCqDBqRw7ou1cpGQJNKuruav4NCZY
66,Tom Davidson,500,tomdavidson92@gmail.com,X3WGSQHHC97PK2KJZ91EBA9HNG,DqEGBoAklSsRtLkfZYrSa8NQ9GUZY
67,Will French,1000,william_french@hotmail.com,BY3NQW9DRX6EF10SFJDHNVEF38,7MdlYXYI7DHcLY7q0aF6nQ0A84fZY


In [85]:
invoices_api = client.invoices

In [86]:
due_at =  str(dt.datetime.now().date())
hour_due=dt.datetime.now().hour
due_at+f'T{hour_due}:00:00Z',

('2024-09-24T10:00:00Z',)

In [87]:
invoices = []

for i, row in order_mg[order_mg['id'].notnull()].iterrows():
    order_id = row['obj_id']
    cust_id = row['id']
    invoice_number = str(dt.datetime.now().date()) +f"_{row['obj_id']}_1"

    

    title = 'Total Outstanding Match Subs'

        
    body = {
    'invoice': {
        'location_id': 'SBBJSGR24YHYB',
        'order_id': f'{order_id}',
        'primary_recipient': {
            'customer_id': f'{cust_id}'
        },
        'payment_requests': [
            {
                'request_type': 'BALANCE',
                'due_date': due_at,
                'tipping_enabled': True,
                'automatic_payment_source': 'NONE',
                'reminders': [
                    {
                        'relative_scheduled_days': -1,
                        'message': 'Your Invoice is due'
                    }
                ]
            }
        ],
        'delivery_method': 'EMAIL',
        'invoice_number': invoice_number,
        'title': title,
        'description': title,
        'scheduled_at': due_at+f'T{hour_due}:00:00Z',
        'accepted_payment_methods': {
            'card': True,
            'square_gift_card': False,
            'bank_account': False,
            'buy_now_pay_later': False,
            'cash_app_pay': False
        },
        'sale_or_service_date': str(dt.datetime.now().date()),
        'store_payment_method_enabled': False
    },
    'idempotency_key': f'{uuid.uuid1()}'
    }

    result = invoices_api.create_invoice(body)
    print(row['email'])
    print(result)
    invidid=result.body['invoice']['id']
    invoices.append(invidid)

byrneyboii2@gmail.com
<ApiResponse {"invoice":{"id":"inv:0-ChDMpK4NAygT6pWl3AJ7NHuyEIMP","version":0,"location_id":"SBBJSGR24YHYB","order_id":"NJHJjp0W7BS60TwRa1f2QPdCOQbZY","payment_requests":[{"uid":"7da81eaf-0751-4ad3-91fb-9dc0b0fc593c","request_type":"BALANCE","due_date":"2024-09-24","tipping_enabled":true,"computed_amount_money":{"amount":500,"currency":"GBP"},"total_completed_amount_money":{"amount":0,"currency":"GBP"},"reminders":[{"uid":"a9ff3df5-b6a5-46c9-b716-6e9f1d7a23b8","relative_scheduled_days":-1,"message":"Your Invoice is due","status":"PENDING"}],"automatic_payment_source":"NONE"}],"primary_recipient":{"customer_id":"ZSBNEMP88D61Z4V4FEM51RHD0W","given_name":"Aidan","family_name":"Byrne","email_address":"byrneyboii2@gmail.com","phone_number":"+447360758784"},"invoice_number":"2024-09-24_NJHJjp0W7BS60TwRa1f2QPdCOQbZY_1","title":"Total Outstanding Match Subs","description":"Total Outstanding Match Subs","scheduled_at":"2024-09-24T10:00:00Z","status":"DRAFT","timezone":"Eu

In [88]:
for invidid in invoices:

    body = {
        'version': 0,
        'idempotency_key': f'{invidid}'
    }

    result = invoices_api.publish_invoice(
        invidid,
        body
    )
    print(result)

<ApiResponse {"invoice":{"id":"inv:0-ChDMpK4NAygT6pWl3AJ7NHuyEIMP","version":1,"location_id":"SBBJSGR24YHYB","order_id":"NJHJjp0W7BS60TwRa1f2QPdCOQbZY","payment_requests":[{"uid":"7da81eaf-0751-4ad3-91fb-9dc0b0fc593c","request_type":"BALANCE","due_date":"2024-09-24","tipping_enabled":true,"computed_amount_money":{"amount":500,"currency":"GBP"},"total_completed_amount_money":{"amount":0,"currency":"GBP"},"reminders":[{"uid":"a9ff3df5-b6a5-46c9-b716-6e9f1d7a23b8","relative_scheduled_days":-1,"message":"Your Invoice is due","status":"NOT_APPLICABLE"}],"automatic_payment_source":"NONE"}],"primary_recipient":{"customer_id":"ZSBNEMP88D61Z4V4FEM51RHD0W","given_name":"Aidan","family_name":"Byrne","email_address":"byrneyboii2@gmail.com","phone_number":"+447360758784"},"invoice_number":"2024-09-24_NJHJjp0W7BS60TwRa1f2QPdCOQbZY_1","title":"Total Outstanding Match Subs","description":"Total Outstanding Match Subs","scheduled_at":"2024-09-24T10:00:00Z","status":"SCHEDULED","timezone":"Europe/London